### <u> _**CRANK-NICOLSON METHOD FOR THE 1D HEAT EQUATION:**_

The Crank-Nicolson method is an implicit finite-difference method to solve partial differential equations such as the heat equation and the likes.
The idea is to compute the field at a given time by averaging the the explicit and the implicit method decomposition of the second derivative in space.

Applied to the 1D heat equation discretized over space and time, denoted by the index $l$ and $j$ we have the following: 

$$
\frac{u^{l+1}_{j} - u^{l}_{j}}{\Delta t} = \frac{D}{2} \frac{ (u^{l}_{j+1}-2u^{l}_{j} + u^{l}_{j+1}) + (u^{l+1}_{j+1}-2u^{l+1}_{j} + u^{+1}_{j+1})}{(\Delta x)^2}
$$

Defining the coefficient $\lambda = \frac{D\Delta t}{(\Delta x)^2}$, we have : 

$$
u^{l+1}_{j} =  u^{l}_{j} + \frac{\lambda}{2}  [ (u^{l}_{j+1}-2u^{l}_{j} + u^{l}_{j+1}) + (u^{l+1}_{j+1}-2u^{l+1}_{j} + u^{+1}_{j+1}) ]
$$

The advantage of this scheme is its perpetual stability, in contrast to the explicit method which is stable only in the case of $\lambda < \frac{1}{2}$.

Re-arranging the term by separating the the knowns($l$) and the unknowns($l+1$) we have: 
 
$$

-\lambda u^{l+1}_{j+1} + 2(1+\lambda) u^{l+1}_{j} - \lambda u^{l+1}_{j-1} = \lambda u^{l}_{j+1}  + 2(1-\lambda) u^{l}_{j} + \lambda u^{l}_{j-1}
$$

Which is a tridiagonal system. Let us consider an example with $j= 1,2,3,4$ we have: 

$$

\begin{bmatrix}         
2(1+\lambda) & -\lambda & 0 & -\lambda\\
 -\lambda    & 2(1+\lambda) & -\lambda & 0\\  
0 & -\lambda & 2(1+\lambda) & -\lambda\\
 -\lambda & 0 & -\lambda & 2(1+\lambda)\\    
\end{bmatrix}

\begin{bmatrix}
u^{l+1}_{1}\\
u^{l+1}_{2}\\
u^{l+1}_{3}\\
u^{l+1}_{4}\\
\end{bmatrix}

 = 
\begin{bmatrix}         
2(1-\lambda) & \lambda & 0 & \lambda\\
 \lambda    & 2(1-\lambda) & \lambda & 0\\  
0 & \lambda & 2(1+\lambda) & \lambda\\
 \lambda & 0 & \lambda & 2(1-\lambda)\\    
\end{bmatrix}

\begin{bmatrix}
0\\
u^{l}_{2} \\
u^{l}_{3}\\
0\\
\end{bmatrix}



$$

To implement this scheme, we just need to compute the matrix beforehand and simply update the right-hand side of the equation at every time step before solving the system of equations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy
from matplotlib import animation
plt.style.use(['ggplot'])

In [3]:
nx = 501
nt = 501

J = np.linspace(0,1,nx)
L = np.linspace(0,10,nt)

dx = 1/(nx-1)
dt = 1/(nt-1)

D = 1 
lbd = (D*dt)/(dx**2) 

u = np.zeros((nt,nx))
BCs = [0,0]
u0 = np.sin(np.pi *J[1:-1]) #np.exp(-100*(J[1:-1]-0.5)**2)

u[:,0] = BCs[0]
u[:,-1] = BCs[1]

u[0,1:-1] = u0


A = np.diag( [2*(1+lbd)]*(nx-2),k = 0) + np.diag( [-lbd]*(nx-3),k = 1) + np.diag( [-lbd]*(nx-3),k = -1)
B = np.diag( [2*(1-lbd)]*(nx-2),k = 0) + np.diag( [lbd]*(nx-3),k = 1) + np.diag( [lbd]*(nx-3),k = -1)


for l in range(nt-1):
    b = u[l,1:-1]
    b = B.dot(b) 
    sol = np.linalg.solve(A,b)
    u[l+1,1:-1] = sol



fig,ax = plt.subplots()

ax.grid(True)
ax.set_xlabel('x [-]')
ax.set_ylabel('Amplitude [-]')

def ani(i):
    ax.clear()
    fig.suptitle(rf'$t ={i*dt:.1f}$')
    ax.set_ylim(0,1)
    ax.plot(J,u[i,:],label = r'$u(x,t)$')
    plt.legend()


%matplotlib qt 

heat_animation = animation.FuncAnimation(fig, ani, frames=nt,interval=40, repeat=True)


heat_animation.save('heat_animation.gif')

MovieWriter ffmpeg unavailable; using Pillow instead.
